In [2]:
import os
import time
import zipfile
import numpy as np
import math


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score,matthews_corrcoef, make_scorer
from sklearn.model_selection import StratifiedKFold



from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints):
    """Performs vectorization using CountVectorizer."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z = countVec.fit_transform(dataPoints)
    return Z



###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = "compressedDataset/flaky_files.zip"
nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

# Create directories
outDirEqual = "results/equal_flaky_nonflaky/"
outDirLarger = "results/larger_nonflaky/"
os.makedirs(outDirEqual, exist_ok=True)
os.makedirs(outDirLarger, exist_ok=True)

extractDirEqual = "extracted/equal_flaky_nonflaky/"
extractDirLarger = "extracted/larger_nonflaky/"
os.makedirs(extractDirEqual, exist_ok=True)
os.makedirs(extractDirLarger, exist_ok=True)

# Extract and read data once for equal combination
flakyDirEqual = os.path.join(extractDirEqual, 'flaky')
nonFlakyDirEqual = os.path.join(extractDirEqual, 'nonFlaky')
os.makedirs(flakyDirEqual, exist_ok=True)
os.makedirs(nonFlakyDirEqual, exist_ok=True)

extract_zip(flakyZip, flakyDirEqual)
extract_zip(nonFlakyZip, nonFlakyDirEqual)

dataPointsFlakyEqual = getDataPoints(flakyDirEqual)
dataPointsNonFlakyEqual = getDataPoints(nonFlakyDirEqual)
dataPointsEqual = dataPointsFlakyEqual + dataPointsNonFlakyEqual

# Print the number of datasets for equal combination
print(f"Number of flaky documents (equal combination): {len(dataPointsFlakyEqual)}")
print(f"Number of non-flaky documents (equal combination): {len(dataPointsNonFlakyEqual)}")
print(f"Total number of documents (equal combination): {len(dataPointsEqual)}")

dataLabelsListEqual = np.array([1]*len(dataPointsFlakyEqual) + [0]*len(dataPointsNonFlakyEqual))

# Vectorize data once
Z_equal = flastVectorization(dataPointsEqual)

print("************SAHPE od DATA:", Z_equal.shape)
### After the split, PCA should happen
### MCC -> make sure way score weighted

# Extract and read data once for larger non-flaky combination
flakyDirLarger = os.path.join(extractDirLarger, 'flaky')
nonFlakyDirLarger = os.path.join(extractDirLarger, 'nonFlaky')
os.makedirs(flakyDirLarger, exist_ok=True)
os.makedirs(nonFlakyDirLarger, exist_ok=True)

extract_zip(flakyZip, flakyDirLarger)
extract_zip(largerNonFlakyZip, nonFlakyDirLarger)

dataPointsFlakyLarger = getDataPoints(flakyDirLarger)
dataPointsNonFlakyLarger = getDataPoints(nonFlakyDirLarger)
dataPointsLarger = dataPointsFlakyLarger + dataPointsNonFlakyLarger

# Print the number of datasets for larger combination
print(f"Number of flaky documents (larger combination): {len(dataPointsFlakyLarger)}")
print(f"Number of non-flaky documents (larger combination): {len(dataPointsNonFlakyLarger)}")
print(f"Total number of documents (larger combination): {len(dataPointsLarger)}")

dataLabelsListLarger = np.array([1]*len(dataPointsFlakyLarger) + [0]*len(dataPointsNonFlakyLarger))

Z_larger = flastVectorization(dataPointsLarger)


Number of flaky documents (equal combination): 45
Number of non-flaky documents (equal combination): 45
Total number of documents (equal combination): 90
************SAHPE od DATA: (90, 7563)
Number of flaky documents (larger combination): 45
Number of non-flaky documents (larger combination): 254
Total number of documents (larger combination): 299


## KNN ##

In [67]:
from sklearn.neighbors import KNeighborsClassifier



### training performance -> 
### return performance for trainning?

def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runKNN(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()
    
    # Define the pipeline with PCA and KNN
    pipeline = Pipeline([
        ('pca', PCA()),  
        ('knn', KNeighborsClassifier())
    ])
    
    
    dataset_length = Z.shape[0]
    
    print ('Data length',dataset_length)
    print([math.floor(i*0.08*dataset_length) for i in range(7,10)])
    
    
    # Parameter grid for hyperparameter tuning
    param_grid = {
        
        'pca__n_components': [math.floor(i*0.08*dataset_length) for i in range (7,10)],  # Variance ratios
        'knn__n_neighbors': [3, 5, 7, 9, 11, 15, 20],
        'knn__metric': ['cosine', 'euclidean'],
        'knn__weights': ['uniform', 'distance'],
    }
    
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),  
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer, #MCC score custom function
    }
    print(type(Z))

    
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline, 
        param_grid, 
        cv=skf, 
        scoring=scoring, 
        refit='f1', 
        verbose=1, 
        return_train_score=True
    )
    
    # Fit the GridSearchCV on data
    grid_search.fit(Z, dataLabelsList)
    
    
    # Get the best parameters and the best score for f1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")
    
    # Save the results
    outFile = f"{combination_label}-params-knn-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("pca_n_components,n_neighbors,metric,weights,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)  
            fo.write(f"{param['pca__n_components']},{param['knn__n_neighbors']},{param['knn__metric']},{param['knn__weights']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")
    
    print(f"KNN analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run KNN on equal combination
print("\nStarting KNN analysis for flaky vs smaller non-flaky files (equal combination)...")
best_params_5folds_knn_equal, best_score_5folds_knn_equal = runKNN(Z_equal, dataLabelsListEqual, outDirEqual, 5, "equal")
best_params_3folds_knn_equal, best_score_3folds_knn_equal = runKNN(Z_equal, dataLabelsListEqual, outDirEqual, 3, "equal")

# Display results
print("\nBest results for KNN 5-fold on equal combination:")
print(f"Best Parameters: {best_params_5folds_knn_equal}")
print(f"Best F1 Score: {best_score_5folds_knn_equal}")

print("\nBest results for KNN 3-fold on equal combination:")
print(f"Best Parameters: {best_params_3folds_knn_equal}")
print(f"Best F1 Score: {best_score_3folds_knn_equal}")

# Run KNN on larger non-flaky combination
print("\nStarting KNN analysis for flaky vs larger non-flaky files (larger combination)...")
best_params_5folds_knn_larger, best_score_5folds_knn_larger = runKNN(Z_larger, dataLabelsListLarger, outDirLarger, 5, "larger")
best_params_3folds_knn_larger, best_score_3folds_knn_larger = runKNN(Z_larger, dataLabelsListLarger, outDirLarger, 3, "larger")

# Display results
print("\nBest results for KNN 5-fold on larger combination:")
print(f"Best Parameters: {best_params_5folds_knn_larger}")
print(f"Best F1 Score: {best_score_5folds_knn_larger}")

print("\nBest results for KNN 3-fold on larger combination:")
print(f"Best Parameters: {best_params_3folds_knn_larger}")
print(f"Best F1 Score: {best_score_3folds_knn_larger}")



Starting KNN analysis for flaky vs smaller non-flaky files (equal combination)...
Data length 90
[50, 57, 64]
<class 'scipy.sparse._csr.csr_matrix'>
Fitting 5 folds for each of 84 candidates, totalling 420 fits
Best Parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'distance', 'pca__n_components': 64}
Best F1 Score: 0.7326984126984126
KNN analysis completed for 5-folds. Results saved to: equal-params-knn-5-folds.csv
Data length 90
[50, 57, 64]
<class 'scipy.sparse._csr.csr_matrix'>
Fitting 3 folds for each of 84 candidates, totalling 252 fits


/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
84 fits failed out of a total of 252.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
84 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 469, in fit
    Xt

Best Parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'distance', 'pca__n_components': 57}
Best F1 Score: 0.7008547008547009
KNN analysis completed for 3-folds. Results saved to: equal-params-knn-3-folds.csv

Best results for KNN 5-fold on equal combination:
Best Parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'distance', 'pca__n_components': 64}
Best F1 Score: 0.7326984126984126

Best results for KNN 3-fold on equal combination:
Best Parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'distance', 'pca__n_components': 57}
Best F1 Score: 0.7008547008547009

Starting KNN analysis for flaky vs larger non-flaky files (larger combination)...
Data length 299
[167, 191, 215]
<class 'scipy.sparse._csr.csr_matrix'>
Fitting 5 folds for each of 84 candidates, totalling 420 fits
Best Parameters: {'knn__metric': 'cosine', 'knn__n_neighbors': 5, 'knn__weights': 'distance', 'pca__n_components': 167}
Best F1 Sco

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
84 fits failed out of a total of 252.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
56 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 469, in fit
    Xt

Best Parameters: {'knn__metric': 'cosine', 'knn__n_neighbors': 7, 'knn__weights': 'distance', 'pca__n_components': 191}
Best F1 Score: 0.5988681849551415
KNN analysis completed for 3-folds. Results saved to: larger-params-knn-3-folds.csv

Best results for KNN 5-fold on larger combination:
Best Parameters: {'knn__metric': 'cosine', 'knn__n_neighbors': 5, 'knn__weights': 'distance', 'pca__n_components': 167}
Best F1 Score: 0.5868131868131868

Best results for KNN 3-fold on larger combination:
Best Parameters: {'knn__metric': 'cosine', 'knn__n_neighbors': 7, 'knn__weights': 'distance', 'pca__n_components': 191}
Best F1 Score: 0.5988681849551415


## SVM

In [72]:
from sklearn.svm import SVC

def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runSVM(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()
    
    # Define the pipeline with PCA and SVM
    pipeline = Pipeline([
        ('pca', PCA()),
        ('svm', SVC())
    ])
    
    dataset_length = Z.shape[0]
    print('Data length', dataset_length)
    print([math.floor(i * 0.08 * dataset_length) for i in range(7, 10)])
    
    # Parameter grid for hyperparameter tuning
    param_grid = {
        'pca__n_components': [math.floor(i * 0.08 * dataset_length) for i in range(7, 10)],
        'svm__C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
        'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel types
    }
    
    # Custom scoring functions including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,  # MCC score custom function
    }
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )
    
    # Fit the GridSearchCV on data
    grid_search.fit(Z, dataLabelsList)
    
    # Get the best parameters and the best score for f1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")
    
    # Save the results
    outFile = f"{combination_label}-params-svm-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("pca_n_components,C,kernel,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            fo.write(f"{param['pca__n_components']},{param['svm__C']},{param['svm__kernel']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")
    
    print(f"SVM analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run SVM on equal combination
print("\nStarting SVM analysis for flaky vs smaller non-flaky files (equal combination)...")
best_params_5folds_svm_equal, best_score_5folds_svm_equal = runSVM(Z_equal, dataLabelsListEqual, outDirEqual, 5, "equal")
best_params_3folds_svm_equal, best_score_3folds_svm_equal = runSVM(Z_equal, dataLabelsListEqual, outDirEqual, 3, "equal")

# Display results
print("\nBest results for SVM 5-fold on equal combination:")
print(f"Best Parameters: {best_params_5folds_svm_equal}")
print(f"Best F1 Score: {best_score_5folds_svm_equal}")

print("\nBest results for SVM 3-fold on equal combination:")
print(f"Best Parameters: {best_params_3folds_svm_equal}")
print(f"Best F1 Score: {best_score_3folds_svm_equal}")

# Run SVM on larger non-flaky combination
print("\nStarting SVM analysis for flaky vs larger non-flaky files (larger combination)...")
best_params_5folds_svm_larger, best_score_5folds_svm_larger = runSVM(Z_larger, dataLabelsListLarger, outDirLarger, 5, "larger")
best_params_3folds_svm_larger, best_score_3folds_svm_larger = runSVM(Z_larger, dataLabelsListLarger, outDirLarger, 3, "larger")

# Display results
print("\nBest results for SVM 5-fold on larger combination:")
print(f"Best Parameters: {best_params_5folds_svm_larger}")
print(f"Best F1 Score: {best_score_5folds_svm_larger}")

print("\nBest results for SVM 3-fold on larger combination:")
print(f"Best Parameters: {best_params_3folds_svm_larger}")
print(f"Best F1 Score: {best_score_3folds_svm_larger}")



Starting SVM analysis for flaky vs smaller non-flaky files (equal combination)...
Data length 90
[50, 57, 64]
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'pca__n_components': 50, 'svm__C': 0.1, 'svm__kernel': 'linear'}
Best F1 Score: 0.8686824905400756
SVM analysis completed for 5-folds. Results saved to: equal-params-svm-5-folds.csv
Data length 90
[50, 57, 64]
Fitting 3 folds for each of 60 candidates, totalling 180 fits


/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 469, in fit
    Xt

Best Parameters: {'pca__n_components': 50, 'svm__C': 100.0, 'svm__kernel': 'rbf'}
Best F1 Score: 0.8146555157307844
SVM analysis completed for 3-folds. Results saved to: equal-params-svm-3-folds.csv

Best results for SVM 5-fold on equal combination:
Best Parameters: {'pca__n_components': 50, 'svm__C': 0.1, 'svm__kernel': 'linear'}
Best F1 Score: 0.8686824905400756

Best results for SVM 3-fold on equal combination:
Best Parameters: {'pca__n_components': 50, 'svm__C': 100.0, 'svm__kernel': 'rbf'}
Best F1 Score: 0.8146555157307844

Starting SVM analysis for flaky vs larger non-flaky files (larger combination)...
Data length 299
[167, 191, 215]
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'pca__n_components': 167, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score: 0.7330065359477125
SVM analysis completed for 5-folds. Results saved to: larger-params-svm-5-folds.csv
Data length 299
[167, 191, 215]
Fitting 3 folds for each of 60 candidates, totalling 1

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 469, in fit
    Xt

Best Parameters: {'pca__n_components': 167, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score: 0.6546469622331692
SVM analysis completed for 3-folds. Results saved to: larger-params-svm-3-folds.csv

Best results for SVM 5-fold on larger combination:
Best Parameters: {'pca__n_components': 167, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score: 0.7330065359477125

Best results for SVM 3-fold on larger combination:
Best Parameters: {'pca__n_components': 167, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score: 0.6546469622331692


## Naive Bayes

In [76]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import time
import math
import os

def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient (MCC).
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runNB(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define the pipeline with PCA and Naive Bayes
    pipeline = Pipeline([
        ('pca', PCA()),
        ('nb', MultinomialNB())
    ])

    dataset_length = Z.shape[0]
    print('Data length', dataset_length)
    print([math.floor(i * 0.08 * dataset_length) for i in range(7, 10)])

    # Parameter grid for hyperparameter tuning
    param_grid = {
        'pca__n_components': [math.floor(i * 0.08 * dataset_length) for i in range(7, 10)],
        'nb__alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
    }

    # Scoring metrics including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for f1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save the results
    outFile = f"{combination_label}-params-nb-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("pca_n_components,alpha,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            fo.write(f"{param['pca__n_components']},{param['nb__alpha']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")

    print(f"Naive Bayes analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run NB on equal combination
print("\nStarting NB analysis for flaky vs smaller non-flaky files (equal combination)...")
best_params_5folds_nb_equal, best_score_5folds_nb_equal = runNB(Z_equal, dataLabelsListEqual, outDirEqual, 5, "equal")
best_params_3folds_nb_equal, best_score_3folds_nb_equal = runNB(Z_equal, dataLabelsListEqual, outDirEqual, 3, "equal")

# Display results
print("\nBest results for NB 5-fold on equal combination:")
print(f"Best Parameters: {best_params_5folds_nb_equal}")
print(f"Best F1 Score: {best_score_5folds_nb_equal}")

print("\nBest results for NB 3-fold on equal combination:")
print(f"Best Parameters: {best_params_3folds_nb_equal}")
print(f"Best F1 Score: {best_score_3folds_nb_equal}")

# Run NB on larger non-flaky combination
print("\nStarting NB analysis for flaky vs larger non-flaky files (larger combination)...")
best_params_5folds_nb_larger, best_score_5folds_nb_larger = runNB(Z_larger, dataLabelsListLarger, outDirLarger, 5, "larger")
best_params_3folds_nb_larger, best_score_3folds_nb_larger = runNB(Z_larger, dataLabelsListLarger, outDirLarger, 3, "larger")

# Display results
print("\nBest results for NB 5-fold on larger combination:")
print(f"Best Parameters: {best_params_5folds_nb_larger}")
print(f"Best F1 Score: {best_score_5folds_nb_larger}")

print("\nBest results for NB 3-fold on larger combination:")
print(f"Best Parameters: {best_params_3folds_nb_larger}")
print(f"Best F1 Score: {best_score_3folds_nb_larger}")





Starting NB analysis for flaky vs smaller non-flaky files (equal combination)...
Data length 90
[50, 57, 64]
Fitting 5 folds for each of 15 candidates, totalling 75 fits


ValueError: 
All the 75 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_meth

## XGB

In [78]:
from xgboost import XGBClassifier


def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runXGB(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define the pipeline with PCA and XGBoost
    pipeline = Pipeline([
        ('pca', PCA()),
        ('xgb', XGBClassifier(eval_metric="logloss", use_label_encoder=False))
    ])

    dataset_length = Z.shape[0]
    print('Data length', dataset_length)
    print([math.floor(i * 0.08 * dataset_length) for i in range(7, 10)])

    # Parameter grid for hyperparameter tuning
    param_grid = {
        'pca__n_components': [math.floor(i * 0.08 * dataset_length) for i in range(7, 10)],
        'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5],  # Learning rate
        'xgb__max_depth': [3, 5, 7, 10],              # Tree depth
        'xgb__n_estimators': [50, 100, 200, 300],     # Number of boosting rounds
    }

    
    
    # Scoring metrics including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,  # MCC score custom function
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save the results
    outFile = f"{combination_label}-params-xgb-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("pca_n_components,learning_rate,max_depth,n_estimators,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            fo.write(f"{param['pca__n_components']},{param['xgb__learning_rate']},{param['xgb__max_depth']},{param['xgb__n_estimators']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")

    print(f"XGBoost analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run XGBoost on equal combination
print("\nStarting XGBoost analysis for flaky vs smaller non-flaky files (equal combination)...")
best_params_5folds_xgb_equal, best_score_5folds_xgb_equal = runXGB(Z_equal, dataLabelsListEqual, outDirEqual, 5, "equal")
best_params_3folds_xgb_equal, best_score_3folds_xgb_equal = runXGB(Z_equal, dataLabelsListEqual, outDirEqual, 3, "equal")

# Display results
print("\nBest results for XGBoost 5-fold on equal combination:")
print(f"Best Parameters: {best_params_5folds_xgb_equal}")
print(f"Best F1 Score: {best_score_5folds_xgb_equal}")

print("\nBest results for XGBoost 3-fold on equal combination:")
print(f"Best Parameters: {best_params_3folds_xgb_equal}")
print(f"Best F1 Score: {best_score_3folds_xgb_equal}")

# Run XGBoost on larger non-flaky combination
print("\nStarting XGBoost analysis for flaky vs larger non-flaky files (larger combination)...")
best_params_5folds_xgb_larger, best_score_5folds_xgb_larger = runXGB(Z_larger, dataLabelsListLarger, outDirLarger, 5, "larger")
best_params_3folds_xgb_larger, best_score_3folds_xgb_larger = runXGB(Z_larger, dataLabelsListLarger, outDirLarger, 3, "larger")

# Display results
print("\nBest results for XGBoost 5-fold on larger combination:")
print(f"Best Parameters: {best_params_5folds_xgb_larger}")
print(f"Best F1 Score: {best_score_5folds_xgb_larger}")

print("\nBest results for XGBoost 3-fold on larger combination:")
print(f"Best Parameters: {best_params_3folds_xgb_larger}")
print(f"Best F1 Score: {best_score_3folds_xgb_larger}")


Starting XGBoost analysis for flaky vs smaller non-flaky files (equal combination)...
Data length 90
[50, 57, 64]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:10:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:11:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

...

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:32:55] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:32:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:32:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:32:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:33:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:33:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:33:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:33:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
192 fits failed out of a total of 576.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
128 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 469, in fit
    

Best Parameters: {'pca__n_components': 167, 'xgb__learning_rate': 0.5, 'xgb__max_depth': 3, 'xgb__n_estimators': 50}
Best F1 Score: 0.49805749805749805
XGBoost analysis completed for 3-folds. Results saved to: larger-params-xgb-3-folds.csv

Best results for XGBoost 5-fold on larger combination:
Best Parameters: {'pca__n_components': 167, 'xgb__learning_rate': 0.5, 'xgb__max_depth': 3, 'xgb__n_estimators': 100}
Best F1 Score: 0.6498701298701299

Best results for XGBoost 3-fold on larger combination:
Best Parameters: {'pca__n_components': 167, 'xgb__learning_rate': 0.5, 'xgb__max_depth': 3, 'xgb__n_estimators': 50}
Best F1 Score: 0.49805749805749805


/Users/gimdongchan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:33:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier

def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runRF(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define the pipeline with PCA and Random Forest
    pipeline = Pipeline([
        ('pca', PCA()),
        ('rf', RandomForestClassifier())
    ])

    dataset_length = Z.shape[0]
    print('Data length', dataset_length)

  
    # Parameter grid for hyperparameter tuning
    param_grid = {
        'pca__n_components': [math.floor(i * 0.08 * dataset_length) for i in range(7, 10)],
        'rf__n_estimators': [10, 50, 100, 300, 500],  # Number of trees
        'rf__max_depth': [10, 30, 50, 100, 300, 500],  # Maximum depth of each tree
        'rf__min_samples_split': [2, 5],  # Minimum number of samples required to split an internal node
        'rf__min_samples_leaf': [1, 2],  # Minimum number of samples required to be at a leaf node
        'rf__criterion': ["gini", "entropy"],  # Function to measure the quality of a split
    }

    # Custom scoring functions including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer  # Use the custom mcc_scorer function
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save the results
    outFile = f"{combination_label}-params-rf-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("pca_n_components,n_estimators,max_depth,min_samples_split,min_samples_leaf,criterion,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            fo.write(f"{param['pca__n_components']},{param['rf__n_estimators']},{param['rf__max_depth']},{param['rf__min_samples_split']},{param['rf__min_samples_leaf']},{param['rf__criterion']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")

    print(f"Random Forest analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run Random Forest on equal combination
print("\nStarting Random Forest analysis for flaky vs smaller non-flaky files (equal combination)...")
best_params_5folds_rf_equal, best_score_5folds_rf_equal = runRF(Z_equal, dataLabelsListEqual, outDirEqual, 5, "equal")
best_params_3folds_rf_equal, best_score_3folds_rf_equal = runRF(Z_equal, dataLabelsListEqual, outDirEqual, 3, "equal")

# Display results
print("\nBest results for Random Forest 5-fold on equal combination:")
print(f"Best Parameters: {best_params_5folds_rf_equal}")
print(f"Best F1 Score: {best_score_5folds_rf_equal}")

print("\nBest results for Random Forest 3-fold on equal combination:")
print(f"Best Parameters: {best_params_3folds_rf_equal}")
print(f"Best F1 Score: {best_score_3folds_rf_equal}")

# Run Random Forest on larger non-flaky combination
print("\nStarting Random Forest analysis for flaky vs larger non-flaky files (larger combination)...")
best_params_5folds_rf_larger, best_score_5folds_rf_larger = runRF(Z_larger, dataLabelsListLarger, outDirLarger, 5, "larger")
best_params_3folds_rf_larger, best_score_3folds_rf_larger = runRF(Z_larger, dataLabelsListLarger, outDirLarger, 3, "larger")

# Display results
print("\nBest results for Random Forest 5-fold on larger combination:")
print(f"Best Parameters: {best_params_5folds_rf_larger}")
print(f"Best F1 Score: {best_score_5folds_rf_larger}")

print("\nBest results for Random Forest 3-fold on larger combination:")
print(f"Best Parameters: {best_params_3folds_rf_larger}")
print(f"Best F1 Score: {best_score_3folds_rf_larger}")



Starting Random Forest analysis for flaky vs smaller non-flaky files (equal combination)...
Data length 90
Fitting 5 folds for each of 720 candidates, totalling 3600 fits


C:\Users\haha9\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'pca__n_components': 64, 'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Best F1 Score: 0.8173076923076923
Random Forest analysis completed for 5-folds. Results saved to: equal-params-rf-5-folds.csv
Data length 90
Fitting 3 folds for each of 720 candidates, totalling 2160 fits


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
720 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, 

Best Parameters: {'pca__n_components': 50, 'rf__criterion': 'gini', 'rf__max_depth': 100, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Best F1 Score: 0.7838433967466226
Random Forest analysis completed for 3-folds. Results saved to: equal-params-rf-3-folds.csv

Best results for Random Forest 5-fold on equal combination:
Best Parameters: {'pca__n_components': 64, 'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Best F1 Score: 0.8173076923076923

Best results for Random Forest 3-fold on equal combination:
Best Parameters: {'pca__n_components': 50, 'rf__criterion': 'gini', 'rf__max_depth': 100, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Best F1 Score: 0.7838433967466226

Starting Random Forest analysis for flaky vs larger non-flaky files (larger combination)...
Data length 299
Fitting 5 folds for each of 720 candidates, totalling 3600 fits

C:\Users\haha9\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'pca__n_components': 167, 'rf__criterion': 'gini', 'rf__max_depth': 100, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 10}
Best F1 Score: 0.5288461538461539
Random Forest analysis completed for 5-folds. Results saved to: larger-params-rf-5-folds.csv
Data length 299
Fitting 3 folds for each of 720 candidates, totalling 2160 fits


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
720 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
480 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, 

Best Parameters: {'pca__n_components': 167, 'rf__criterion': 'gini', 'rf__max_depth': 100, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 10}
Best F1 Score: 0.4192232887885062
Random Forest analysis completed for 3-folds. Results saved to: larger-params-rf-3-folds.csv

Best results for Random Forest 5-fold on larger combination:
Best Parameters: {'pca__n_components': 167, 'rf__criterion': 'gini', 'rf__max_depth': 100, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 10}
Best F1 Score: 0.5288461538461539

Best results for Random Forest 3-fold on larger combination:
Best Parameters: {'pca__n_components': 167, 'rf__criterion': 'gini', 'rf__max_depth': 100, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 10}
Best F1 Score: 0.4192232887885062


## Decision Tree

In [6]:
from sklearn.tree import DecisionTreeClassifier

def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runDT(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define the pipeline with PCA and Decision Tree
    pipeline = Pipeline([
        ('pca', PCA()),
        ('dt', DecisionTreeClassifier())
    ])

    dataset_length = Z.shape[0]
    print('Data length', dataset_length)

  

    # Parameter grid for hyperparameter tuning
    param_grid = {
        'pca__n_components': [math.floor(i * 0.08 * dataset_length) for i in range(7, 10)],
        'dt__criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
        'dt__max_depth': [None, 10, 30, 50, 100, 300, 500],  # Maximum depth of the tree
        'dt__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
        'dt__min_samples_leaf': [1, 2, 5, 10],  # Minimum number of samples required to be at a leaf node
        'dt__max_features': [None, 'sqrt', 'log2'],  # Number of features to consider when looking for the best split
    }

    # Scoring metrics including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,  # MCC score custom function
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save the results
    outFile = f"{combination_label}-params-dt-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("pca_n_components,criterion,max_depth,min_samples_split,min_samples_leaf,max_features,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            fo.write(f"{param['pca__n_components']},{param['dt__criterion']},{param['dt__max_depth']},{param['dt__min_samples_split']},{param['dt__min_samples_leaf']},{param['dt__max_features']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")

    print(f"Decision Tree analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run Decision Tree on equal combination
print("\nStarting Decision Tree analysis for flaky vs smaller non-flaky files (equal combination)...")
best_params_5folds_dt_equal, best_score_5folds_dt_equal = runDT(Z_equal, dataLabelsListEqual, outDirEqual, 5, "equal")
best_params_3folds_dt_equal, best_score_3folds_dt_equal = runDT(Z_equal, dataLabelsListEqual, outDirEqual, 3, "equal")

# Display results
print("\nBest results for Decision Tree 5-fold on equal combination:")
print(f"Best Parameters: {best_params_5folds_dt_equal}")
print(f"Best F1 Score: {best_score_5folds_dt_equal}")

print("\nBest results for Decision Tree 3-fold on equal combination:")
print(f"Best Parameters: {best_params_3folds_dt_equal}")
print(f"Best F1 Score: {best_score_3folds_dt_equal}")

# Run Decision Tree on larger non-flaky combination
print("\nStarting Decision Tree analysis for flaky vs larger non-flaky files (larger combination)...")
best_params_5folds_dt_larger, best_score_5folds_dt_larger = runDT(Z_larger, dataLabelsListLarger, outDirLarger, 5, "larger")
best_params_3folds_dt_larger, best_score_3folds_dt_larger = runDT(Z_larger, dataLabelsListLarger, outDirLarger, 3, "larger")

# Display results
print("\nBest results for Decision Tree 5-fold on larger combination:")
print(f"Best Parameters: {best_params_5folds_dt_larger}")
print(f"Best F1 Score: {best_score_5folds_dt_larger}")

print("\nBest results for Decision Tree 3-fold on larger combination:")
print(f"Best Parameters: {best_params_3folds_dt_larger}")
print(f"Best F1 Score: {best_score_3folds_dt_larger}")



Starting Decision Tree analysis for flaky vs smaller non-flaky files (equal combination)...
Data length 90
Fitting 5 folds for each of 1512 candidates, totalling 7560 fits
Best Parameters: {'dt__criterion': 'gini', 'dt__max_depth': 100, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 10, 'pca__n_components': 50}
Best F1 Score: 0.8029399585921325
Decision Tree analysis completed for 5-folds. Results saved to: equal-params-dt-5-folds.csv
Data length 90
Fitting 3 folds for each of 1512 candidates, totalling 4536 fits


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1512 fits failed out of a total of 4536.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1512 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y

Best Parameters: {'dt__criterion': 'entropy', 'dt__max_depth': 500, 'dt__max_features': None, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 2, 'pca__n_components': 50}
Best F1 Score: 0.7821664925113202
Decision Tree analysis completed for 3-folds. Results saved to: equal-params-dt-3-folds.csv

Best results for Decision Tree 5-fold on equal combination:
Best Parameters: {'dt__criterion': 'gini', 'dt__max_depth': 100, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 10, 'pca__n_components': 50}
Best F1 Score: 0.8029399585921325

Best results for Decision Tree 3-fold on equal combination:
Best Parameters: {'dt__criterion': 'entropy', 'dt__max_depth': 500, 'dt__max_features': None, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 2, 'pca__n_components': 50}
Best F1 Score: 0.7821664925113202

Starting Decision Tree analysis for flaky vs larger non-flaky files (larger combination)...
Data length 299
Fitting 5 folds for each of 1512 candidates, totallin

C:\Users\haha9\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 5, 'pca__n_components': 167}
Best F1 Score: 0.5517126148705096
Decision Tree analysis completed for 5-folds. Results saved to: larger-params-dt-5-folds.csv
Data length 299
Fitting 3 folds for each of 1512 candidates, totalling 4536 fits
Best Parameters: {'dt__criterion': 'gini', 'dt__max_depth': 30, 'dt__max_features': 'log2', 'dt__min_samples_leaf': 10, 'dt__min_samples_split': 10, 'pca__n_components': 167}
Best F1 Score: 0.5395061728395062
Decision Tree analysis completed for 3-folds. Results saved to: larger-params-dt-3-folds.csv

Best results for Decision Tree 5-fold on larger combination:
Best Parameters: {'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 5, 'pca__n_components': 167}
Best F1 Score: 0.5517126148705096

Best results for Decision Tree 3-fold on 

C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1512 fits failed out of a total of 4536.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1008 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y